In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium

# For real distance
import osmnx as ox
import networkx as nx

import src.cleaning as cleaning

In [2]:
# Sample to test functions:
apr_14 = pd.read_csv('../../../Desktop/final_project_data/2014/2014-04 - Citi Bike trip data.csv')

# rename columns

In [7]:
apr_14 = cleaning.rename_columns(apr_14)

#  Straight line Distance

In [8]:
apr_14 = cleaning.trip_distance(apr_14)

#  start_hour & end_hour (same function, 2 columns)

- integer value for the hour, e.g. 12, 18, etc.

In [9]:
apr_14 = cleaning.get_hour (apr_14)

# trip_date
- time series date format for date, including year, month and day, and excluding hour, minute and second.

In [10]:
apr_14 = cleaning.get_date (apr_14)

# started_at and ended_at in datetime format

In [11]:
apr_14 = cleaning.datetime_format (apr_14)

#  month, weekday, weekend

In [12]:
apr_14 = cleaning.get_categorical_date (apr_14)

In [15]:
apr_14.to_csv('data/april_2014.csv', index=False)

In [2]:
apr_14 = pd.read_csv('data/april_2014.csv')

# real_distance

In [3]:
apr_14_sample = apr_14.sample(100)

In [4]:
apr_14_sample = cleaning.get_real_distance (apr_14_sample)

# rideable_type

(670780, 21)

# trip_cost ()

¡You need to get bike type before!

- float $ value in function of rideable_type, duration and member_casual.

Subscriber
- Classic.
If trip_duration < 45*60, cost = 0
Elif trip_duration > 45*60, cost = (trip_duration - 45 * 60) / 60 * 0.17

- Electric
If trip_duration =< 45*60: cost = trip_duration / 60 * 0.17, limit 3.
If trip_duration > 45*60: cost = 3 + (trip_duration - 45 * 60) / 60 * 0.17

Casual
- Single trip 
    - Casual & - Electric
    If trip_duration <= 30*60 -> 4.49
    Elif trip_duration > 30 * 60 -> 4.49 + (trip_duration - 30 * 60) / 60 * 0.26

- Day Pass
    - Casual 
    - Electric

# Station non bike trips balance

In [15]:
def station_balance (df, station_name, day):
    unique_days = df.trip_date.unique()
    balance_mensual_estacion = []
    for day in unique_days:
        llegadas = apr_14[(df['end_station_name'] == station_name) & (df['trip_date'] == day)].shape[0]
        salidas = apr_14[(df['start_station_name'] == station_name) & (df['trip_date'] == day)].shape[0]
        balance_mensual_estacion.append(llegadas - salidas)
    
    return balance_mensual_estacion

In [5]:
def bike_not_transported (df, bike_id, date):
    '''
    If bike is not transported return true
    '''
    bike_trips = df[(df['bike_id'] == bike_id) & df['trip_date'] == date]
    start_list = []
    end_list = []
    for i, row in bike_trips.iterrows():  
        start_list.append(row['start_station_name'])
        end_list.append(row['end_station_name'])

    start_list = start_list[1:]
    end_list = end_list[:-1]

    if start_list == end_list:
        return True
    else:
        return False

### Start here

In [171]:
def bike_route (df, bike_id):
    bike_trips = df[df['bike_id'] == bike_id]
    bike_journey = []
    start_station = []
    end_station = []

    for index, row in bike_trips.iterrows():
        start_station.append(row['start_station_name'])
        end_station.append(row['end_station_name'])

    start_station = start_station[1:]
    end_station = end_station[:-1]

    for i in range(len(start_station)):
        next_start, last_end = start_station[i], end_station[i]
        bike_journey.append((next_start, last_end))
        
    return bike_journey

In [197]:
def all_bikes_journey (df):
    bikes_journey_list = []
    for bike_id in df['bike_id'].unique():
        bikes_journey_list.append(bike_route (df, bike_id))
    
    return bikes_journey_list

In [198]:
all_bikes_journey_list = all_bikes_journey (apr_14)

In [202]:
def non_trip_mobility_dict (df):
    list_of_start_stations = apr_14.start_station_name.unique().tolist()
    list_of_end_stations = apr_14.end_station_name.unique().tolist()
    all_stations = list(set(list_of_start_stations + list_of_end_stations))
    mobility_dictionary = {}

    for station_name in all_stations:
        if station_name in list_of_start_stations:
            mobility_dictionary[station_name] = {
                'id': df[df['start_station_name'] == station_name].iloc[0]['start_station_id'], 
                'receives_from': [],
                'sends_to': [],             
            }
        elif station_name in list_of_end_stations:
            mobility_dictionary[station_name] = {
                'id': df[df['end_station_name'] == station_name].iloc[0]['end_station_id'], 
                'receives_from': [],
                'sends_to': []                
            }           

    return mobility_dictionary

In [203]:
stations_transfers_dictionary = non_trip_mobility_dict (apr_14)

In [207]:
def transportations (dictionary, bikes_journey_list):
    for bike_journey in bikes_journey_list:
        for arrival_departure in bike_journey:
            if arrival_departure[0] != arrival_departure[1]:
                dictionary[arrival_departure[1]]['sends_to'].append(arrival_departure[0])
                dictionary[arrival_departure[0]]['receives_from'].append(arrival_departure[1])

    return dictionary

In [208]:
stations_transfers_dictionary = transportations (stations_transfers_dictionary, all_bikes_journey_list)

In [209]:
def station_balance (dictionary):
    for key, value in dictionary.items():
        value['bikes_received'] = len(value['receives_from'])
        value['bikes_sent'] = len(value['sends_to'])
        value['balance'] = (value['bikes_received'] - value['bikes_sent']) 
        # if receives more than sends (balance > 0): station is likely to be a start point
        # else if sends more than receives (balance < 0): station is likely to be an end point
    
    return dictionary

In [210]:
stations_transfers_dictionary = station_balance (stations_transfers_dictionary)

In [212]:
stations_transfers_dictionary['West St & Chambers St']

{'id': 426,
 'receives_from': ['E 32 St & Park Ave',
  'Pearl St & Hanover Square',
  'W 24 St & 7 Ave',
  'Vesey Pl & River Terrace',
  'W 52 St & 11 Ave',
  'W 24 St & 7 Ave',
  'Vesey Pl & River Terrace',
  'W 29 St & 9 Ave',
  'Broadway & E 22 St',
  'Vesey Pl & River Terrace',
  'W 31 St & 7 Ave',
  '6 Ave & Canal St',
  'W 53 St & 10 Ave',
  'Harrison St & Hudson St',
  'Broadway & W 58 St',
  'W 45 St & 6 Ave',
  'University Pl & E 14 St',
  '9 Ave & W 45 St',
  'Duane St & Greenwich St',
  'E 47 St & Park Ave',
  'Hudson St & Reade St',
  'Greenwich St & N Moore St',
  'Grand St & Havemeyer St',
  'Broadway & W 55 St',
  'South End Ave & Liberty St',
  'Pearl St & Hanover Square',
  'Broadway & W 60 St',
  'Bedford Ave & S 9th St',
  'Water - Whitehall Plaza',
  'W 14 St & The High Line',
  'Barrow St & Hudson St',
  'Mott St & Prince St',
  'Hudson St & Reade St',
  'W 47 St & 10 Ave',
  'Broadway & W 60 St',
  'Vesey Pl & River Terrace',
  'W 34 St & 11 Ave',
  'LaGuardia Pl 

# Truck trips

Record all bike movements not attributable to user trips.
- Bike ID
- Date range
- Transported from
- Transported to

Sort by date.

In [151]:
apr_14['started_at'][0]

'2014-04-01 00:00:07'

In [235]:
def single_bike_truck_transfers (df, bike_id):
    bike_trips = df[df['bike_id'] == bike_id]
    last_end_station_name = []
    last_end_station_id = []
    last_end_time = []
    next_start_station_name = []
    next_start_station_id = []
    next_start_time = []


    for i, row in bike_trips.iterrows():
        last_end_station_id.append(row['end_station_id'])
        last_end_station_name.append(row['end_station_name'])
        last_end_time.append(row['ended_at'])
        next_start_station_id.append(row['start_station_id'])
        next_start_station_name.append(row['start_station_name'])
        next_start_time.append(row['started_at'])

    last_end_station_id = last_end_station_id[:-1]
    last_end_station_name = last_end_station_name[:-1]
    last_end_time = last_end_time[:-1]

    next_start_station_id = next_start_station_id[1:]
    next_start_station_name = next_start_station_name[1:]
    next_start_time = next_start_time[1:]

    # Definitive lists
    end_id = []
    end_name = []
    end_time = []
    start_id = []
    start_name =[]
    start_time = []
    bike_id_list = []

    for i in range(len(last_end_station_name)):
        if last_end_station_id[i] != next_start_station_id[i]:
            end_id.append(last_end_station_id[i])
            end_name.append(last_end_station_name[i])
            end_time.append(last_end_time[i])
            start_id.append(next_start_station_id[i])
            start_name.append(next_start_station_name[i])
            start_time.append(next_start_time[i])
            bike_id_list.append(bike_id)
    
    transfers_df = pd.DataFrame({
        'last_end_station_id': end_id, 
        'last_end_station_name': end_name,
        'last_end_time': end_time,
        'next_start_station_id': start_id,
        'next_start_station_name': start_name,
        'next_start_time': start_time,
        'bike_id': bike_id_list
    })

    return transfers_df

In [228]:
single_bike_truck_transfers (apr_14, 21062)

,last_end_station_id,last_end_station_name,last_end_time,next_start_station_id,next_start_station_name,next_start_time,bike_id
0,360,William St & Pine St,2014-04-01 08:58:20,306,Cliff St & Fulton St,2014-04-02 15:25:04,21062
1,473,Rivington St & Chrystie St,2014-04-03 12:46:45,522,E 51 St & Lexington Ave,2014-04-05 11:56:28,21062
2,275,Washington Ave & Greene Ave,2014-04-06 12:32:44,431,Hanover Pl & Livingston St,2014-04-09 08:25:44,21062
3,432,E 7 St & Avenue A,2014-04-09 18:06:00,469,Broadway & W 53 St,2014-04-09 21:11:42,21062
4,127,Barrow St & Hudson St,2014-04-10 20:04:32,358,Christopher St & Greenwich St,2014-04-11 06:04:04,21062
5,458,11 Ave & W 27 St,2014-04-11 20:46:33,457,Broadway & W 58 St,2014-04-12 12:14:42,21062
6,345,W 13 St & 6 Ave,2014-04-12 12:37:19,3002,South End Ave & Liberty St,2014-04-12 19:10:59,21062
7,238,Bank St & Washington St,2014-04-16 13:09:11,510,W 51 St & 6 Ave,2014-04-17 17:04:15,21062
8,151,Cleveland Pl & Spring St,2014-04-19 17:27:35,293,Lafayette St & E 8 St,2014-04-19 18:15:22,21062
9,119,Park Ave & St Edwards St,2014-04-22 16:44:12,412,Forsyth St & Canal St,2014-04-23 08:35:21,21062


In [236]:
def all_transfers (df):
    dataframes_list = []
    for bike_id in df['bike_id'].unique():
        dataframes_list.append(single_bike_truck_transfers (df, bike_id))
    
    return dataframes_list

In [238]:
list_of_dataframes = all_transfers (apr_14)

In [243]:
list_of_dataframes[645]

,last_end_station_id,last_end_station_name,last_end_time,next_start_station_id,next_start_station_name,next_start_time,bike_id
0,477,W 41 St & 8 Ave,2014-04-01 16:08:06,465,Broadway & W 41 St,2014-04-01 18:16:11,20871
1,2004,6 Ave & Broome St,2014-04-03 15:45:58,519,E 42 St & Vanderbilt Ave,2014-04-10 06:28:41,20871
2,526,E 33 St & 5 Ave,2014-04-10 14:31:24,325,E 19 St & 3 Ave,2014-04-10 17:48:44,20871
3,328,Watts St & Greenwich St,2014-04-11 11:04:55,472,E 32 St & Park Ave,2014-04-13 16:01:50,20871
4,446,W 24 St & 7 Ave,2014-04-14 14:29:13,444,Broadway & W 24 St,2014-04-14 15:07:55,20871
5,540,Lexington Ave & E 26 St,2014-04-15 09:21:29,174,E 25 St & 1 Ave,2014-04-16 06:39:08,20871
6,2023,E 55 St & Lexington Ave,2014-04-16 06:55:43,494,W 26 St & 8 Ave,2014-04-16 14:48:22,20871
7,250,Lafayette St & Jersey St,2014-04-18 12:55:32,151,Cleveland Pl & Spring St,2014-04-19 14:57:27,20871
8,282,Kent Ave & S 11 St,2014-04-22 18:56:59,532,S 5 Pl & S 4 St,2014-04-23 14:34:13,20871
9,402,Broadway & E 22 St,2014-04-23 22:59:37,509,9 Ave & W 22 St,2014-04-24 07:07:30,20871


In [ ]:
all_bikes_journey_list = all_bikes_journey (apr_14)

In [231]:
a = pd.DataFrame({'a':[1,2], 'b':[2,4]})
b = pd.DataFrame({'a':[1,4], 'b':[8,4]})
c = pd.DataFrame({'a':[4,7], 'b':[1,5]})

In [233]:
dataframes_list = [a, b, c]

In [234]:
dataframes_list[0]

,a,b
0,1,2
1,2,4
